In [11]:
import numpy as np, cv2, glob, time, itertools, scipy
from operator import itemgetter

# Parameters
tf, qf, Na, St, Tt, Ct, Tf, Td, bsize = 4, .25, 3, 3, 0.06, 3, 50, 40, 8
# transform
# tf = 4

# qf = .25
# Na = 3
# St = 3
# Tt = 0.06

# cosine transform?
# Ct = 3
# Tf = 50
# Td = 40

# block size
# bsize = 8

# zz_mat: matrix for the zigzag sort, which element is used next
zz_mat = [0,0, 0,1, 1,0, 2,0, 1,1, 0,2, 0,3, 1,2, 2,1, 3,0, 4,0, 3,1, 2,2, 1,3, 0,4, 0,5, 1,4, 2,3, 3,2, 4,1, 5,0, 6,0, 5,1, 4,2, 3,3, 2,4, 1,5, 0,6, 0,7, 1,6, 2,5, 3,4, 4,3, 5,2, 6,1, 7,0]
zzidx = [0,0, 0,1, 1,0, 2,0, 1,1, 0,2, 0,3, 1,2, 2,1, 3,0, 3,1, 2,2, 1,3, 2,3, 3,2, 3,3]


# DCT on disjoint blocks of size B
def dct_blocks(img, B):

    # get height and width of original image: h = 533, w = 800
    h, w = img.shape

    # convert img values to datatype float
    img = img.astype(float)

    # creates overlapping blocks
    blocks = np.lib.stride_tricks.as_strided(img, shape=(h - B + 1, w - B +1, B, B), strides=img.strides + img.strides)
    print(blocks)

    # img.strides = 6400, 8
    print(img.strides)

    # img.strides + img.strides = 6400, 8, 6400, 8
    print(img.strides + img.strides)

    #
    bh, bw = blocks.shape[:2]
    print(bh, bw)

    # blocks.shape = (526, 793, 8, 8)
    print(blocks.shape)

    i = 1
    j = 2
    myblock = cv2.dct(blocks[i][j]) * 0.25

    print(f"myblock: ", myblock)

    # for every block a cv2.dct is performed
    # all elements are multiplied by 0.25 and added 0.5
    # they are then sorted by a zigzag pattern which is usual for dct
    # and then stored and returned by variable dct

    # is divided by 4: end resultat is 16 instead of 64 ...? CHECK THIS
    dct = [[(i, j), zigzag_sort((cv2.dct(blocks[i][j]) * qf + .5).astype(int).tolist())] for (i,j) in itertools.product(range(bh), range(bw))]
    print(type(dct[10]))
    print(len(dct))
    print((dct[10]))

    # dct is a list with length 417.118 = 526 * 793 = bh * bw
    # it contains first the block location and then the zigzag_sort
    return dct


# does the 1D zigzag_sort of a 2D block
def zigzag_sort(a):
    b = []
    # tf = 4
    # range(0, 2*16, 2) = range(0, 32, 2) in 2er Schritten ergibt 16 Werte
    # 2er Schritte, um immer reihen und spalten wert der matrix zu erhalten
    #
    for i in range(0, 2 * tf**2, 2):
        b.append(a[zz_mat[i]][zz_mat[i + 1]])
    return b


def get_image_results(img, tv):
    img_res = np.zeros(img.shape, dtype = 'uint8')
    s = sorted(tv, key = lambda k: len(tv[k]), reverse=True)[:4]
    for k in s:
        v = tv[k]
        if len(v) > Tf:
            for x in v:
                img_res[x[0], x[1]] = 1
                img_res[x[0] + k[0], x[1] + k[1]] = 1
    return img_res


def compare(A, tv):
    if A == None or len(A) == 0:
        return
    a = A[0]
    for b in A[1:]:
        ai, aj = np.array(a[1]), np.array(b[1])
        idxi, idxj = np.array(a[0]), np.array(b[0])
        v = (idxj[0] - idxi[0], idxj[1] - idxi[1])
        if np.linalg.norm(v) > Td:
            rmax, rmin = -5000, 5000
            c = 0
            for l in range(len(aj)):
                if aj[l] == 0:
                    if abs(ai[l] - aj[l]) >= St:
                        c += 1
                else:
                    r = 1.0*ai[l] / aj[l]
                    if r < rmin:
                        rmin = r
                    if r > rmax:
                        rmax = r
            if rmax - rmin >= Tt:
                c += 1
            if c < Ct:
                idx = idxi
                if v[0] < 0:
                    v = (-v[0], -v[1])
                    idx = idxj
                if tv.get(v) == None:
                    tv[v] = []
                tv[v].append(idx)


# DCT and main frequency extraction
def locate_copy_move_region_dct(img):

    # start timer
    start = time.time()

    # function divides img into dct-performable blocks, with default blocksize = 8, but in return each part is 16
    # elements long
    # A stores the DCT 1D list with 417.118 elements
    A = dct_blocks(img, bsize)
    #print(A)
    print('DCT: {} seconds'.format(time.time() - start))

    # start timer again
    start = time.time()

    # function sorts 1D blocks in A in a descending order (value in total of the block?)
    A = sorted(A, key=itemgetter(1), reverse=True)
    #print(A)
    print('SORT: {} seconds'.format(time.time() - start))

    # start timer again
    start = time.time()

    # Create empty Dictionary of transfer vectors
    tv = {}

    # It creates a list from 0 to len(A) - Na - 1, which will include all possible sublists of length Na = 3
    # Na = 3
  #  AA = list(map(lambda i: A[i: i+Na], list(range(len(A) - Na))))
    #print(AA)
    print('NEIGH: {} seconds'.format(time.time() - start))

    # start timer again
    start = time.time()

    # ???
  #  list(map(lambda X: compare(X, tv), AA))
    print('COMPARE: {} seconds'.format(time.time() - start))

    # start timer again
    start = time.time()

    # original image and dict of transfer vectors are given to calculate image results which are stores in img_res
 #   img_res = get_image_results(img, tv)
    print('RESULTS: {} seconds'.format(time.time() - start))

    # give back the image with found forgeries
    return #img_res



# main:
# read image in grayscale from path and store it in img variable
test_img = 'images/giraffe_copy.png'
img = cv2.imread(test_img, 0)

# perform copy move detection
result = locate_copy_move_region_dct(img)

# write result image to pathcv2.imwrite('images/giraffe_copy-result.png', result*255)

[[[[106. 110. 114. ... 125. 129. 131.]
   [112. 117. 120. ... 131. 136. 137.]
   [117. 122. 127. ... 137. 141. 142.]
   ...
   [134. 141. 146. ... 156. 157. 156.]
   [137. 144. 151. ... 160. 160. 158.]
   [143. 149. 156. ... 166. 165. 163.]]

  [[110. 114. 115. ... 129. 131. 129.]
   [117. 120. 122. ... 136. 137. 141.]
   [122. 127. 128. ... 141. 142. 149.]
   ...
   [141. 146. 150. ... 157. 156. 161.]
   [144. 151. 155. ... 160. 158. 160.]
   [149. 156. 160. ... 165. 163. 163.]]

  [[114. 115. 118. ... 131. 129. 132.]
   [120. 122. 125. ... 137. 141. 142.]
   [127. 128. 132. ... 142. 149. 151.]
   ...
   [146. 150. 153. ... 156. 161. 162.]
   [151. 155. 157. ... 158. 160. 161.]
   [156. 160. 163. ... 163. 163. 164.]]

  ...

  [[145. 145. 140. ... 130. 129. 128.]
   [140. 139. 137. ... 128. 127. 128.]
   [136. 135. 134. ... 126. 127. 127.]
   ...
   [127. 124. 123. ... 124. 129. 132.]
   [123. 120. 119. ... 127. 132. 136.]
   [124. 122. 120. ... 130. 136. 141.]]

  [[145. 140. 137. ..

In [13]:
zz_mat = [0,0, 0,1, 1,0, 2,0, 1,1, 0,2, 0,3, 1,2, 2,1, 3,0, 4,0, 3,1, 2,2, 1,3, 0,4, 0,5, 1,4, 2,3, 3,2, 4,1, 5,0, 6,0, 5,1, 4,2, 3,3, 2,4, 1,5, 0,6, 0,7, 1,6, 2,5, 3,4, 4,3, 5,2, 6,1, 7,0]
len(zz_mat)

72